In [24]:
import pandas as pd
from itertools import combinations
import networkx as nx
import numpy as np
from collections import defaultdict,deque

In [25]:
ruta = r"C:\Users\hamga\Downloads\ClientMovementsSocialNetwork.csv"
df = pd.read_csv(ruta)
print(df.head())

          MES  CLIENTEID MOVIMIENTO  SALDO_IN  SALDO_OUT  AGENTID      CANAL  \
0  31/12/2018          1         in  25777382          0    24597  CHANNEL 4   
1  28/02/2018          2        out         0          0    13087  CHANNEL 5   
2  31/01/2017          3        out         0   17552962    18024  CHANNEL 5   
3  31/08/2019          4         in   1965188          0    14667  CHANNEL 1   
4  31/03/2017          5        out         0   29433839    30677  CHANNEL 5   

   aux  IN  OUT  COMPETIDOR TRANSFIERE  COMPETIDOR RECIBE  
0    1   1    0                     14                  9  
1    1   0    1                      9                  1  
2    1   0    1                      9                 13  
3    1   1    0                      2                  9  
4    1   0    1                      9                 10  


In [26]:
aseguradoras = {
    1: "AXA",
    2: "GNP",
    3: "MAPFRE",
    4: "HDI",
    5: "Zurich",
    6: "Quálitas",
    7: "Inbursa",
    8: "ANA Seguros",
    9: "Chubb",          
    10: "Banorte Seguros",
    11: "Afirme Seguros",
    12: "Seguros Atlas",
    13: "AIG",
    14: "Sura"
}

canales = {
    "CHANNEL 1": "Agente Digital",
    "CHANNEL 2": "Agente Tradicional",
    "CHANNEL 3": "Broker",
    "CHANNEL 4": "Call Center",
    "CHANNEL 5": "Sucursal",
    "CHANNEL 6": "Banca Seguros",
    "CHANNEL 7": "Alianza Comercial",
    "CHANNEL 8": "Otro Canal"
}

for col in ["COMPETIDOR TRANSFIERE", "COMPETIDOR RECIBE"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        df[col] = df[col].map(aseguradoras).fillna(df[col].astype(str))

if "CANAL" in df.columns:
    df["CANAL"] = df["CANAL"].map(canales).fillna(df["CANAL"])

In [27]:
df.head(10)

,MES,CLIENTEID,MOVIMIENTO,SALDO_IN,SALDO_OUT,AGENTID,CANAL,aux,IN,OUT,COMPETIDOR TRANSFIERE,COMPETIDOR RECIBE
0,31/12/2018,1,in,25777382,0,24597,Call Center,1,1,0,Sura,Chubb
1,28/02/2018,2,out,0,0,13087,Sucursal,1,0,1,Chubb,AXA
2,31/01/2017,3,out,0,17552962,18024,Sucursal,1,0,1,Chubb,AIG
3,31/08/2019,4,in,1965188,0,14667,Agente Digital,1,1,0,GNP,Chubb
4,31/03/2017,5,out,0,29433839,30677,Sucursal,1,0,1,Chubb,Banorte Seguros
5,30/09/2016,6,out,0,6352446,19040,Sucursal,1,0,1,Chubb,ANA Seguros
6,31/05/2018,7,out,0,11948542,14204,Sucursal,1,0,1,Chubb,GNP
7,28/02/2019,8,out,0,20660099,15200,Sucursal,1,0,1,Chubb,AIG
8,30/09/2018,9,in,11529358,0,25310,Call Center,1,1,0,MAPFRE,Chubb
9,31/07/2018,10,out,0,15530571,18637,Sucursal,1,0,1,Chubb,MAPFRE


In [28]:
# convertimos la fecha a datetime y ordenamos
df["MES"] = pd.to_datetime(df["MES"], dayfirst=True, errors="coerce")
df = df.sort_values(["CLIENTEID", "MES"])

# solo clientes con min 3 movimientos y sin duplicar agente por cliente
d = df[df.groupby("CLIENTEID")["CLIENTEID"].transform("size") >= 3]
d = d.drop_duplicates(["CLIENTEID", "AGENTID"])

# combinaciones de agentes por cliente
comb = (
    d.groupby("CLIENTEID")["AGENTID"]
     .apply(lambda s: list(combinations(sorted(s.unique()), 2)))
     .explode()
     .dropna()
     .apply(pd.Series)
     .rename(columns={0: "Agente1", 1: "Agente2"})
)

# contar intensidad (# clientes en común)
combinaciones = (
    comb.groupby(["Agente1", "Agente2"])
         .size()
         .reset_index(name="Intensidad")
         .sort_values("Intensidad", ascending=False)
         .reset_index(drop=True)
)

combinaciones[["Agente1","Agente2"]] = (
    combinaciones[["Agente1","Agente2"]]
    .apply(pd.to_numeric, errors="coerce")
)

# 2) forzar orden: menor -> Agente1, mayor -> Agente2
mins = combinaciones[["Agente1","Agente2"]].min(axis=1)
maxs = combinaciones[["Agente1","Agente2"]].max(axis=1)
combinaciones["Agente1"] = mins
combinaciones["Agente2"] = maxs

# 3) colapsar A–B y B–A en un solo registro, sumando Intensidad
combinaciones = (
    combinaciones.groupby(["Agente1","Agente2"], as_index=False)["Intensidad"].sum()
    .sort_values("Intensidad", ascending=False)
    .reset_index(drop=True)
)

# 4) Aux si la necesitas
combinaciones["Aux"] = 1

In [29]:
combinaciones.head(10)

,Agente1,Agente2,Intensidad,Aux
0,16558,25272,14,1
1,6098,21662,9,1
2,17700,23944,9,1
3,17700,30621,8,1
4,8410,30621,8,1
5,16483,21856,8,1
6,8409,8464,7,1
7,15169,25957,7,1
8,7004,21856,7,1
9,22799,27141,6,1


In [30]:
combinaciones.to_csv("Relaciones_Agentes.csv", index=False)

In [31]:

# LO SIGUIENTE ES PARA ARREGLAR EL ISSUE DE LAS NETWORKS EN POWERBI QUE SI UN AGENTE ESTA EN AGENTE1 SERA UN NODO Y SI ESTA EN AGENTE2 ESTA EN OTRO NODO:
# SE HACE CREANDO UN GRAFO CON NETWORKX Y DE AHI VA NODO POR NODO ASIGNANDOLE YA SEA AGENTE1 O AGENTE2 PRIORIZANDO LOS DE MAYOR CENTRALIDAD.

edges = combinaciones[["Agente1","Agente2","Intensidad"]].copy()

for c in ["Agente1","Agente2"]:
    edges[c] = edges[c].astype(str).str.strip().str.replace(r"\.0$", "", regex=True)

edges = edges[edges["Agente1"] != edges["Agente2"]].copy()

uv = np.sort(edges[["Agente1","Agente2"]].to_numpy(), axis=1)
E = (pd.DataFrame(uv, columns=["u","v"])
     .assign(Intensidad=edges["Intensidad"].to_numpy())
     .groupby(["u","v"], as_index=False)["Intensidad"].sum())

# ========= 1) Grafo + prioridad por degree y preferencia de columna =========
G = nx.from_pandas_edgelist(E, "u", "v", edge_attr="Intensidad", create_using=nx.Graph())
deg = dict(G.degree())

# Preferencia por columna según frecuencia en la tabla original
cnt1 = edges.groupby("Agente1").size()  # veces que salió como Agente1
cnt2 = edges.groupby("Agente2").size()  # veces que salió como Agente2

def pref_side(n):
    a1 = int(cnt1.get(n, 0))
    a2 = int(cnt2.get(n, 0))
    return 0 if a1 >= a2 else 1  # 0 = Agente1, 1 = Agente2

def pref_strength(n):
    return abs(int(cnt1.get(n,0)) - int(cnt2.get(n,0)))  # qué tan clara es la preferencia

def edge_priority(a, b):
    # Primero el mayor degree de la arista, luego el segundo.
    # (Si quieres ponderar por preferencia, podrías sumar 0.001*max(pref_strength))
    return (max(deg.get(a,0), deg.get(b,0)),
            min(deg.get(a,0), deg.get(b,0)))

prio = E.apply(lambda r: edge_priority(r["u"], r["v"]), axis=1)
E_sorted = E.loc[prio.sort_values(ascending=False).index].reset_index(drop=True)

# ========= 2) Asignación a dos sets con recoloreo local, respetando preferencia =========
side = {}  # nodo -> 0 (Agente1) o 1 (Agente2)

def flip_component(start, pivot_priority):
    """
    Recolorea (flip 0<->1) la componente alcanzable desde 'start'
    sólo atravesando nodos con degree <= pivot_priority.
    Evita mover nodos fuertes (alto degree).
    """
    q = deque([start])
    visited = set([start])
    while q:
        x = q.popleft()
        side[x] = 1 - side.get(x, 0)
        for y in G.neighbors(x):
            if y in visited:
                continue
            if deg.get(y,0) <= pivot_priority:
                visited.add(y)
                q.append(y)

for _, row in E_sorted.iterrows():
    a, b = row["u"], row["v"]
    sa, sb = side.get(a), side.get(b)

    if sa is None and sb is None:
        # Semilla: ancla al nodo más fuerte en SU lado preferido
        H = a if deg.get(a,0) >= deg.get(b,0) else b
        L = b if H == a else a
        side[H] = pref_side(H)
        side[L] = 1 - side[H]

    elif sa is None and sb is not None:
        # Asigna a 'a' al lado opuesto de 'b' (la arista debe cruzar lados)
        side[a] = 1 - sb

    elif sa is not None and sb is None:
        side[b] = 1 - sa

    else:
        # Ambos asignados: si caen en el mismo set, recolorea al "menos protegido"
        if sa == sb:
            align_a = (sa == pref_side(a))
            align_b = (sb == pref_side(b))

            # Regla: si uno está alineado a su preferencia y el otro no, mueve al no alineado.
            if align_a and not align_b:
                mover = b
            elif align_b and not align_a:
                mover = a
            else:
                # Si ambos alineados (o ambos no alineados): mueve al de menor degree;
                # si empatan, al de menor pref_strength.
                if deg.get(a,0) < deg.get(b,0):
                    mover = a
                elif deg.get(b,0) < deg.get(a,0):
                    mover = b
                else:
                    mover = a if pref_strength(a) <= pref_strength(b) else b

            pivot_prio = deg.get(mover, 0)
            if mover not in side:
                side[mover] = 0
            flip_component(mover, pivot_prio)

# Asegura lados para aislados
for n in G.nodes():
    if n not in side:
        side[n] = pref_side(n)  # incluso aquí respetamos preferencia

# ========= 3) Orientar todas las aristas: set 0 -> set 1 =========
rows = []
still_conflicts = 0
for a, b, w in E[["u","v","Intensidad"]].itertuples(index=False, name=None):
    sa, sb = side[a], side[b]
    if sa == sb:
        # Caso residual raro: fuerza orientación local manteniendo al más fuerte en su preferencia
        still_conflicts += 1
        # Mantén como Agente1 al que tenga mayor degree; si empatan, al de mayor pref_strength;
        # si sigue el empate, al que prefiera lado 0.
        cand = [(a, deg.get(a,0), pref_strength(a), 1 if pref_side(a)==0 else 0),
                (b, deg.get(b,0), pref_strength(b), 1 if pref_side(b)==0 else 0)]
        cand.sort(key=lambda t: (-t[1], -t[2], -t[3], str(t[0])))
        keep = cand[0][0]
        other = b if keep == a else a
        Ag1, Ag2 = (keep, other) if side.get(keep,0) == 0 else (other, keep)
    else:
        Ag1, Ag2 = (a, b) if sa == 0 and sb == 1 else (b, a)
    rows.append((Ag1, Ag2, w, 1))

out = (pd.DataFrame(rows, columns=["Agente1","Agente2","Intensidad","Aux"])
         .groupby(["Agente1","Agente2"], as_index=False)["Intensidad"].sum()
         .sort_values("Intensidad", ascending=False)
         .reset_index(drop=True))

out["aux"]=1

out.to_csv("Agentes_Bipartito_Recolor_Preferido.csv", index=False)

print(f"Generado 'Agentes_Bipartito_Recolor_Preferido.csv' con {len(out)} aristas.")
print(f"Conflictos residuales forzados (sin cambiar side global): {still_conflicts}")

Generado 'Agentes_Bipartito_Recolor_Preferido.csv' con 1297 aristas.
Conflictos residuales forzados (sin cambiar side global): 288


# PT 2

In [ ]:
# 1) Asegurar aristas únicas no dirigidas (A-B == B-A, sin duplicados)
uv = np.sort(combinaciones[["Agente1","Agente2"]].values, axis=1)
edges_u = pd.DataFrame(uv, columns=["Agente1","Agente2"]).drop_duplicates()

# 2) Grafo simple (una arista por par)
G = nx.from_pandas_edgelist(edges_u, source="Agente1", target="Agente2", create_using=nx.Graph())

# 3) Métricas por nodo
degree    = dict(G.degree())                 # vecinos únicos
closeness = nx.closeness_centrality(G)       # ya viene normalizada en NetworkX

# 4) DataFrame final y riesgo
out = pd.DataFrame({
    "Agente": list(G.nodes()),
    "Degree_raw": [degree[n] for n in G.nodes()],
    "Closeness":  [closeness[n] for n in G.nodes()]
})
out["Riesgo"] = out["Degree_raw"] * out["Closeness"]

# (opcional) ordenar
out = out.sort_values("Riesgo", ascending=False).reset_index(drop=True)

# --- NUEVO: solo Media + σ ---
media = out["Riesgo"].mean()
std   = out["Riesgo"].std(ddof=1)
umbral_ms = media + std

out["RiesgoNivel"] = np.where(out["Riesgo"] > umbral_ms, "Alto", "Bajo/Medio")
pct_alto_ms = 100 * (out["Riesgo"] > umbral_ms).mean()

# Guardar outputs
out.to_csv("Centralidades_Riesgo_etiquetado.csv", index=False)

📈 % de agentes con riesgo ALTO (Media+σ): 5.42%
✅ Listo: Centralidades_Riesgo_etiquetado.csv
